In [ ]:
pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install langchain openai beautifulsoup4 requests

In [ ]:
pip install --upgrade langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [ ]:
from google.colab import userdata
import os

#os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_AI_KEY")

from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

1. Research Agent

In [ ]:
import requests
from bs4 import BeautifulSoup

def research_agent(state):
    query = state["query"]
    print(f"🔍 Researching: {query}")

    search_url = f"https://www.google.com/search?q={query}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"}

    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    results = []
    for g in soup.find_all('h3'):
        if g.text:
            results.append(g.text)

    return {"raw_data": results[:5]}  # Return the top 5 results


In [ ]:
import os
os.environ['OPEN_API_KEY'] = "OPEN_AI_KEY"

2. Summarisation Agent

In [ ]:
import os
# Correct the environment variable name
os.environ["OPENAI_API_KEY"] = "OPEN_AI_KEY" # Replace "YOUR_ACTUAL_OPENAI_API_KEY" with your real OpenAI key

from posix import environ
from langchain.chat_models import ChatOpenAI

# Alternatively, you can pass the key directly to ChatOpenAI
# llm = ChatOpenAI(openai_api_key="YOUR_ACTUAL_OPENAI_API_KEY", model="gpt-4")
llm = ChatOpenAI(model="gpt-4")

def summarization_agent(state):
    raw_data = state["raw_data"]
    print("📝 Summarizing research data...")

    summary_prompt = f"Summarize the following search results:\n\n{raw_data}"
    summary = llm.invoke(summary_prompt).content

    return {"summary": summary}

<ipython-input-7-ec571ac8eaa0>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4")


In [ ]:
import os
# Correct the environment variable name
os.environ["OPENAI_API_KEY"] = "OPEN_AI_KEY" # Replace "YOUR_ACTUAL_OPENAI_API_KEY" with your real OpenAI key

from posix import environ
from langchain.chat_models import ChatOpenAI

# Alternatively, you can pass the key directly to ChatOpenAI
# llm = ChatOpenAI(openai_api_key="YOUR_ACTUAL_OPENAI_API_KEY", model="gpt-4")
llm = ChatOpenAI(model="gpt-4")

def summarization_agent(state):
    raw_data = state["raw_data"]
    print("📝 Summarizing research data...")

    summary_prompt = f"Summarize the following search results:\n\n{raw_data}"
    summary = llm.invoke(summary_prompt).content

    return {"summary": summary}

3. Sentiment Analysis

In [ ]:
def sentiment_analysis_agent(state):
    summary = state["summary"]
    print("📊 Analyzing sentiment...")

    sentiment_prompt = f"Determine if the following text has a positive, neutral, or negative sentiment:\n\n{summary}"
    sentiment = llm.invoke(sentiment_prompt).content

    return {"sentiment": sentiment}

4. Report Generator

In [ ]:
def report_generator(state):
    summary = state["summary"]
    sentiment = state["sentiment"]

    report = f"""
    📌 **Research Summary**
    {summary}

    📊 **Sentiment Analysis**
    Sentiment: {sentiment}
    """

    return {"final_report": report}

## Building the LangGraph Workflow

In [ ]:
from langgraph.graph import StateGraph, END

# Create the graph
graph = StateGraph(dict)  # Use dict as the state format

# Add nodes (functions) with proper return values
graph.add_node("research", research_agent)  # Explicitly specify output keys
graph.add_node("summarization", summarization_agent)
graph.add_node("sentiment_analysis", sentiment_analysis_agent)
graph.add_node("report", report_generator)

# Define the workflow edges
graph.set_entry_point("research")
graph.add_edge("research", "summarization")
graph.add_edge("summarization", "sentiment_analysis")
graph.add_edge("sentiment_analysis", "report")
graph.add_edge("report", END)  # Define the termination point

# Compile the workflow
app = graph.compile()

In [ ]:
from IPython.display import Image, display
try:
  display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [ ]:
# Run the assistant with a research query
query = "Impact of AI on job market"
result = app.invoke({"query":query})
# Print the final report
print(result["final_report"])

🔍 Researching: Impact of AI on job market
📝 Summarizing research data...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPEN_AI_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}